In [8]:
import pandas as pd
from sodapy import Socrata
import numpy as np 
import ast
import json

CommunityToPoliceDistrict = []
with open('PoliceDistrictToCommunityNamesOutput.txt') as fp:
    for line in fp:
        communityNum, communityName, policeDist  = line.strip().split(":")
        communityName = communityName.strip()
        CommunityToPoliceDistrict.append({"Community Area": float(communityNum), "Community_Name": communityName,
                                         "Police_District": policeDist})

CommunityPoliceDistrictDF = pd.DataFrame(CommunityToPoliceDistrict)
CommunityPoliceDistrictDF.head()

,Community Area,Community_Name,Police_District
0,1.0,ROGERS PARK,24
1,2.0,WEST RIDGE,24
2,3.0,UPTOWN,23
3,4.0,LINCOLN SQUARE,20
4,5.0,NORTH CENTER,19


In [9]:
CrimeData = pd.read_csv("Crimes2001_To_Feb2020.csv",
                       dtype={'ID': int, 'Case Number' : str, 'Date' : str, 'Block' : str, 
                             'IUCR' : str, 'Primary Type' : str, 'Description' : str, 
                             'Location Description' : str, 'Arrest' : str, 'Domestic' : str, 'Beat' : str, 
                             'District' : float, 'Ward' : float, 'Community Area' : float, 'FBI Code' : str, 
                              'X Coordinate' : str, 'Y Coordinate' : str, 'Year' : int,'Updated On' : str,
                             'Latitude' : str, 'Longitude' : str, 'Location' : str})

CrimeData = CrimeData.drop(['X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude', 'Location'], axis = 1)
First100CrimeData = CrimeData[:100]
First100CrimeData = First100CrimeData.join(CommunityPoliceDistrictDF.set_index('Community Area'), on='Community Area')
print(First100CrimeData.head())
print(First100CrimeData.columns)
print(First100CrimeData['Community Area'].unique())

         ID Case Number                    Date                 Block  IUCR  \
0  11034701    JA366925  01/01/2001 11:00:00 AM       016XX E 86TH PL  1153   
1  11227287    JB147188  10/08/2017 03:00:00 AM    092XX S RACINE AVE  0281   
2  11227583    JB147595  03/28/2017 02:00:00 PM       026XX W 79TH ST  0620   
3  11227293    JB147230  09/09/2017 08:17:00 PM  060XX S EBERHART AVE  0810   
4  11227634    JB147599  08/26/2017 10:00:00 AM   001XX W RANDOLPH ST  0281   

          Primary Type                          Description  \
0   DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT OVER $ 300   
1  CRIM SEXUAL ASSAULT                       NON-AGGRAVATED   
2             BURGLARY                       UNLAWFUL ENTRY   
3                THEFT                            OVER $500   
4  CRIM SEXUAL ASSAULT                       NON-AGGRAVATED   

  Location Description Arrest Domestic  Beat  District  Ward  Community Area  \
0            RESIDENCE  false    false  0412       4.0   8.0      

In [10]:
# Used to Extract the data from the Socio Economic CSV File 
# To train our program what specific communities are better or worse off than average
import pandas as pd 
import numpy as np 

# We can use .describe() to find the benchmark levels of poverty per community. Compare community to the average and give weightage 
columns = [
'Community Area',
'community_area_name', 
'%_housing_crowded', 
'%_households_below_poverty', 
'%_age16_unemployed',
'%_age25_nohighschool',
'%_not_working_age',
'per_capita_income',
'hardship_index']

SocioeconomicData = pd.read_csv("SocioeconomicData.csv")
SocioeconomicData.columns = columns

First100CrimeData = First100CrimeData.join(SocioeconomicData.set_index('Community Area'), on='Community Area')

print(First100CrimeData.columns)

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'Year', 'Updated On',
       'Community_Name', 'Police_District', 'community_area_name',
       '%_housing_crowded', '%_households_below_poverty', '%_age16_unemployed',
       '%_age25_nohighschool', '%_not_working_age', 'per_capita_income',
       'hardship_index'],
      dtype='object')


In [11]:
import pandas as pd

census = pd.read_csv("census_data.csv", header=1)

CensusData = pd.DataFrame(census)
CensusData = CensusData[CensusData.columns[range(67)]]
CensusData = CensusData.rename(columns={"Geog": "Community Name", "GeogKey": "Community Area"})

#First100CrimeData = First100CrimeData.join(CensusData.set_index('Community Area'), on='Community Area')
First100CrimeData = First100CrimeData.merge(CensusData, on='Community Area', how='left')
print(First100CrimeData.columns)
print(First100CrimeData.head())

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'Year', 'Updated On',
       'Community_Name', 'Police_District', 'community_area_name',
       '%_housing_crowded', '%_households_below_poverty', '%_age16_unemployed',
       '%_age25_nohighschool', '%_not_working_age', 'per_capita_income',
       'hardship_index', 'Community Name', 'Total Population',
       'Not Hispanic or Latino, White alone',
       'Not Hispanic or Latino, Black or African American alone',
       'Not Hispanic or Latino, American Indian and Alaska Native alone',
       'Not Hispanic or Latino, Asian alone',
       'Not Hispanic or Latino, Native Hawaiian and Other Pacific Islander alone',
       'Not Hispanic or Latino, Some Other Race alone',
       'Not Hispanic or Latino, Two or More Races', 'Hispanic or Latino',
       'Male: Under 5 years old', 'Male: 5 to 9 

In [47]:
from datetime import datetime, time
morningStart = time(6, 0, 0)
morningEnd = time(12,0,0)
afternoonStart = time(12,0,1)
afternoonEnd = time(18,30,0)
nightStart = time(18,31,0)
nightEnd = time(23,59,59)
midnightStart = time(0,0,0)
midnightEnd = time(5,59,0)

# Time of Day Function
def timeOfDayApplier(dateColumn):
    DatetimeVersion = datetime.strptime(dateColumn,'%m/%d/%Y %I:%M:%S %p') # 'I' accounts for 12-hour format
    
    if morningStart <= DatetimeVersion.time() <= morningEnd:
        return "Morning"
    elif afternoonStart <= DatetimeVersion.time() <= afternoonEnd:
        return "Afternoon/Evening"
    elif nightStart <= DatetimeVersion.time() <= nightEnd:
        return "Night"
    elif midnightStart <= DatetimeVersion.time() <= midnightEnd:
        return "Midnight"

#timeOfDayApplier(First100CrimeData['Date'])
First100CrimeData['TimeOfDay'] = First100CrimeData['Date'].map(timeOfDayApplier)


In [51]:
from datetime import datetime, time
springStart = 3
springEnd = 5
summerStart = 6
summerEnd = 8
fallStart = 9
fallEnd = 11
winterStart = 12
winterEnd = 2 
# Time of Day Function
def seasonOfYear(dateColumn):
    DatetimeVersion = datetime.strptime(dateColumn,'%m/%d/%Y %I:%M:%S %p') # 'I' accounts for 12-hour format
     
    if DatetimeVersion.month in [12,1,2]:
        return "Winter"   
    elif springStart <= DatetimeVersion.month <= springEnd:
        return "Spring"
    elif summerStart <= DatetimeVersion.month <= summerEnd:
        return "Summer"
    elif fallStart <= DatetimeVersion.month <= fallEnd:
        return "Fall"

First100CrimeData['Season'] = First100CrimeData['Date'].map(seasonOfYear)

In [52]:
print(First100CrimeData['Date'][0]) # value check 
print(First100CrimeData['TimeOfDay'][0]) # value check
print(First100CrimeData['Season'][0]) # value check

01/01/2001 11:00:00 AM
Morning
Winter
